In [2]:
import pandas as pd
import numpy as np

In [3]:
data= pd.read_csv('auteurs_mouvements.csv')

In [14]:
data

,Unnamed: 0,Nom,Mouvement,summary,sum_lem,mvm_label
0,0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0
1,1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0
2,2,Carolyn Cassady,Beat Generation,"Carolyn Cassady, née Carolyn Robinson le 28 av...",Carolyn Cassady naître Carolyn Robinson 28 avr...,0
3,3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0
4,4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0
...,...,...,...,...,...,...
256,256,Maurice Beaubourg,symboliste,"Maurice Beaubourg, né le 4 décembre 1859 et mo...",Maurice Beaubourg naître 4 décembre 1859 mort ...,9
257,257,Otokar Březina,symboliste,"Otokar Březina, né Václav Ignác Jebavý le 13 s...",otokar Březina naître Václav Ignác jebavý 13 s...,9
258,258,Emanuil Popdimitrov,symboliste,"Emanuil Popdimitrov Popzahariev est un poète, ...",emanuil Popdimitrov Popzahariev poète philosop...,9
259,259,Marcel Schwob,symboliste,"Marcel Schwob, né à Chaville (Seine-et-Oise, a...",Marcel Schwob naître Chaville 23 août 1867 mor...,9


In [4]:
datab = pd.read_csv('Mouvements_wiki_cleaned.csv')

In [ ]:
#je recopie les fonctions de prédiction que je modifie quelque peu (rajout option 2) 

In [5]:
#juste pour ressortir toutes les prédictions "ex aequo" dans la fonction suivante

def sort_ish(L):
    mov,key = L[0]
    sorted_final=[mov]
    k=1
    n=len(L)
    while k<n:
        mov,keyy=L[k]
        if keyy<key:
            return sorted_final
        else:
            sorted_final.append(mov)
            k+=1
    return sorted_final

In [52]:
#j'ai fixé les arguments sauf name pour pvoir utiliser directement sur le dataframe avec la méthode apply (cf ci dessous)
#c'est évidemment juste pour le premier essai

# name : nom de l'auteur
#database : base de données contenant tous les auteurs
#train : base de données content tous les mouvements
# n : si tf-idf, nombre de mots utilisés dans la liste tf idf
# option 1 : tf-idf pour prédire OU texte entier du mouvement 'tf-idf' ou 'texte'
# option2 : 'sum' ou 'whole' pour savoir si l'on prend pour l'auteur les données du texte de son summary ou du texte de la page entière


def pred(name,database,train,n,option1,option2):
    if option1=='tf-idf':
        #avec l'option tf-idf, on se base sur les mots les plus importants pour chaque mouvement au sens 
        #de la métrique tf-idf
        if option2=='sum':
            txt = database[database['Nom']==name]['sum_lem'].values[0].split()
        elif option2=='whole':
            txt = database[database['Nom']==name]['page_lem'].values[0].split()
            
        #on prend le texte associé à l'auteur *name*
        
        pred={}
        #dictionnaire des prédictions
        
        m=len(train)
        
        for k in range(m):
            #pour chaque mouvement
            
            mov = train['mouvement'][k] #on récupère son nom
            L= eval(train['tf_idf'][k])[:n] #et les n premiers mots tf_idf (n paramètre de la fction)
            c=0 #compteur qui va compter combien de mots du texte associé à l'auteur (le summary wikipedia)
            #sont des mots présents dans les n tf idf
            for keyword in L:
                #pour chaque mot clé tf idf
                if keyword in txt:
                    #s'il est dans le texte associé à l'auteur, on incrémente c.
                    c+=1
            pred[mov]=c 
            #le nombre de mots est associé au mouvement dans le dictionnaire
        PRED = sorted(pred.items(), key=lambda mot_proba: mot_proba[1])      
        PRED.reverse()
        #on ordonne le dictionnaire selon les clés
        return sort_ish(PRED) #on retourne tous les mots ayant la valeur maximale ! 
    
    
    if option1=='texte':
        #idem que pour tf-idf sauf qu'on s'intéresse à tout le texte lemmatisé de la page du mouvement 
        if option2=='sum':
            txt = database[database['Nom']==name]['sum_lem'].values[0].split()
        elif option2=='whole':
            txt = database[database['Nom']==name]['page_lem'].values[0].split()
        pred={}
        m=len(train)
        for k in range(m):
            mov = train['mouvement'][k]
            L= train['texte_lem'][k].split() #voilà la seule différence : 
            #on prend comme liste de mots L le texte_lem qui est le texte de la page wiki du mouvement.
            c=0
            for keyword in L:
                if keyword in txt:
                    c+=1
            pred[mov]=c
        PRED = sorted(pred.items(), key=lambda mot_proba: mot_proba[1])
        PRED.reverse()
        return sort_ish(PRED)


In [7]:
#data['pred_txt']=data['Nom'].apply(pred)

TypeError: pred() missing 5 required positional arguments: 'database', 'train', 'n', 'option1', and 'option2'

In [25]:
#data['pred_txt']=data['pred_txt'].apply(lambda L: [aut_clean(L[0])])

Vraies prédictions

In [8]:
data = pd.read_csv('auteurs_completed_lem.csv',index_col=[0])

In [9]:
import wikipedia 
from wikipedia import WikipediaPage
wikipedia.set_lang("fr")

In [81]:
data_acompleter = data[data['is_empty']==0]

In [11]:
data_complete = data[data['is_empty']==1]

In [83]:
def wiki(s):
    return WikipediaPage(s).content

In [84]:
data_acompleter['page']=data_acompleter['Nom'].apply(wiki)

<ipython-input-84-3591b3dfc3c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_acompleter['page']=data_acompleter['Nom'].apply(wiki)


In [86]:
data_acompleter.to_csv('auteurs_incomplete.csv')

In [14]:
data_acompleter=pd.read_csv('auteurs_incomplete_completed.csv',index_col=[0])

In [16]:
auteurs = pd.concat([data_complete,data_acompleter])

In [18]:
auteurs.to_csv('auteurs_complete2.csv')

In [26]:
mouvements=pd.read_csv('Mouvements_wiki_cleaned1.csv',index_col=[0])

In [20]:
del auteurs['is_empty']
auteurs['len']=auteurs['page_lem'].apply(len)

In [24]:
authors = list(auteurs['Nom'])

In [29]:
from tqdm import tqdm

In [47]:
database=auteurs
train=mouvements
n=50

In [44]:
m=len(authors)
predictions = [[] for k in range(m)]

In [74]:


OPT1 = ['tf-idf','texte']
OPT2 = ['whole','sum']
for k in tqdm(range(m)):
    L=[]
    for i in range(2):
        for j in range(2):
            name = authors[k]
            option1 = OPT1[i]
            option2 = OPT2[j]
            a=pred(name,database,train,n,option1,option2)
            L.append(a)
    predictions[k]=L
            
            
            
        

100%|████████████████████████████████████████████████████████████████████████████████| 622/622 [12:37<00:00,  1.22s/it]


In [75]:
auteurs['pred']=predictions

In [76]:
auteurs

,Nom,Mouvement,summary,sum_lem,mvm_label,page,page_lem,len,pred
0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0,"William Seward Burroughs II (; February 5, 191...",William Seward Burroughs ii February 5 1914 au...,61923,"[[Beat Generation], [Beat Generation], [symbol..."
1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0,"Lucien Carr (March 1, 1925 – January 28, 2005)...",Lucien Carr March 1 1925 January 28 2005 was a...,18339,"[[Beat Generation], [Beat Generation, Générati..."
3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0,"John Clellon Holmes (March 12, 1926, Holyoke, ...",John Clellon Holmes March 12 1926 Holyoke Mass...,2592,"[[Beat Generation], [Génération perdue], [Beat..."
4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0,"Gregory Nunzio Corso (March 26, 1930 – January...",Gregory Nunzio Corso March 26 1930 January 17 ...,40570,"[[Beat Generation], [Beat Generation], [symbol..."
5,Lenore Kandel,Beat Generation,"Lenore Kandel, née le 14 janvier 1932 à New Yo...",Lenore Kandel naître 14 janvier 1932 New York ...,0,"Lenore Kandel (January 14, 1932 in New York Ci...",Lenore Kandel January 14 1932 in New York City...,5128,"[[Beat Generation], [Beat Generation], [Beat G..."
...,...,...,...,...,...,...,...,...,...
573,Louis_Guillaume,Ecole de Rochefort,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,2,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,5062,"[[Parnasse], [pleiade], [symboliste], [symboli..."
576,Gabriel_Audisio_(romancier_et_poète),Ecole de Rochefort,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,2,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,5139,"[[Ecole de Rochefort], [Imagisme, pleiade], [s..."
595,Jean-Vincent_Verdonnet,Ecole de Rochefort,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2041,"[[Ecole de Rochefort], [pleiade], [symboliste]..."
598,Alain_Robbe-Grillet,Nouveau Roman,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,8,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,7079,"[[Nouveau Roman], [Nouveau Roman], [symboliste..."


In [77]:
auteurs['pred_tf_w']=auteurs['pred'].apply(lambda L: L[0])
auteurs['pred_tf_sum']=auteurs['pred'].apply(lambda L: L[1])
auteurs['pred_txt_w']=auteurs['pred'].apply(lambda L: L[2])
auteurs['pred_txt_s']=auteurs['pred'].apply(lambda L: L[3])

In [79]:
auteurs[['Mouvement','pred_tf_w','pred_tf_sum','pred_txt_w','pred_txt_s']]

,Mouvement,pred_tf_w,pred_tf_sum,pred_txt_w,pred_txt_s
0,Beat Generation,[Beat Generation],[Beat Generation],[symboliste],[symboliste]
1,Beat Generation,[Beat Generation],"[Beat Generation, Génération perdue]",[symboliste],[symboliste]
3,Beat Generation,[Beat Generation],[Génération perdue],[Beat Generation],[symboliste]
4,Beat Generation,[Beat Generation],[Beat Generation],[symboliste],[Beat Generation]
5,Beat Generation,[Beat Generation],[Beat Generation],[Beat Generation],[Beat Generation]
...,...,...,...,...,...
573,Ecole de Rochefort,[Parnasse],[pleiade],[symboliste],[symboliste]
576,Ecole de Rochefort,[Ecole de Rochefort],"[Imagisme, pleiade]",[symboliste],[symboliste]
595,Ecole de Rochefort,[Ecole de Rochefort],[pleiade],[symboliste],[symboliste]
598,Nouveau Roman,[Nouveau Roman],[Nouveau Roman],[symboliste],[symboliste]


In [55]:
auteurs['pred']=predictions

In [58]:
auteurs

,Nom,Mouvement,summary,sum_lem,mvm_label,page,page_lem,len,pred
0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0,"William Seward Burroughs II (; February 5, 191...",William Seward Burroughs ii February 5 1914 au...,61923,[[symboliste]]
1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0,"Lucien Carr (March 1, 1925 – January 28, 2005)...",Lucien Carr March 1 1925 January 28 2005 was a...,18339,[[symboliste]]
3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0,"John Clellon Holmes (March 12, 1926, Holyoke, ...",John Clellon Holmes March 12 1926 Holyoke Mass...,2592,[[symboliste]]
4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0,"Gregory Nunzio Corso (March 26, 1930 – January...",Gregory Nunzio Corso March 26 1930 January 17 ...,40570,[[Beat Generation]]
5,Lenore Kandel,Beat Generation,"Lenore Kandel, née le 14 janvier 1932 à New Yo...",Lenore Kandel naître 14 janvier 1932 New York ...,0,"Lenore Kandel (January 14, 1932 in New York Ci...",Lenore Kandel January 14 1932 in New York City...,5128,[[Beat Generation]]
...,...,...,...,...,...,...,...,...,...
573,Louis_Guillaume,Ecole de Rochefort,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,2,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,5062,[[symboliste]]
576,Gabriel_Audisio_(romancier_et_poète),Ecole de Rochefort,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,2,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,5139,[[symboliste]]
595,Jean-Vincent_Verdonnet,Ecole de Rochefort,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2041,[[symboliste]]
598,Alain_Robbe-Grillet,Nouveau Roman,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,8,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,7079,[[symboliste]]


In [81]:
mouvements_auteurs = list(set(auteurs.Mouvement.values))

In [85]:
L=[]
for mov in mouvements_auteurs:
    if mov not in list(set(mouvements.mouvement)):
        L.append(mov)

In [87]:
auteurs['del']=auteurs.Mouvement.apply(lambda mov: int(mov in L))

In [90]:
AUT

,Nom,Mouvement,summary,sum_lem,mvm_label,page,page_lem,len,pred,pred_tf_w,pred_tf_sum,pred_txt_w,pred_txt_s,del
0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0,"William Seward Burroughs II (; February 5, 191...",William Seward Burroughs ii February 5 1914 au...,61923,"[[Beat Generation], [Beat Generation], [symbol...",[Beat Generation],[Beat Generation],[symboliste],[symboliste],0
1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0,"Lucien Carr (March 1, 1925 – January 28, 2005)...",Lucien Carr March 1 1925 January 28 2005 was a...,18339,"[[Beat Generation], [Beat Generation, Générati...",[Beat Generation],"[Beat Generation, Génération perdue]",[symboliste],[symboliste],0
3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0,"John Clellon Holmes (March 12, 1926, Holyoke, ...",John Clellon Holmes March 12 1926 Holyoke Mass...,2592,"[[Beat Generation], [Génération perdue], [Beat...",[Beat Generation],[Génération perdue],[Beat Generation],[symboliste],0
4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0,"Gregory Nunzio Corso (March 26, 1930 – January...",Gregory Nunzio Corso March 26 1930 January 17 ...,40570,"[[Beat Generation], [Beat Generation], [symbol...",[Beat Generation],[Beat Generation],[symboliste],[Beat Generation],0
5,Lenore Kandel,Beat Generation,"Lenore Kandel, née le 14 janvier 1932 à New Yo...",Lenore Kandel naître 14 janvier 1932 New York ...,0,"Lenore Kandel (January 14, 1932 in New York Ci...",Lenore Kandel January 14 1932 in New York City...,5128,"[[Beat Generation], [Beat Generation], [Beat G...",[Beat Generation],[Beat Generation],[Beat Generation],[Beat Generation],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Louis_Guillaume,Ecole de Rochefort,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,2,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,5062,"[[Parnasse], [pleiade], [symboliste], [symboli...",[Parnasse],[pleiade],[symboliste],[symboliste],0
576,Gabriel_Audisio_(romancier_et_poète),Ecole de Rochefort,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,2,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,5139,"[[Ecole de Rochefort], [Imagisme, pleiade], [s...",[Ecole de Rochefort],"[Imagisme, pleiade]",[symboliste],[symboliste],0
595,Jean-Vincent_Verdonnet,Ecole de Rochefort,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2041,"[[Ecole de Rochefort], [pleiade], [symboliste]...",[Ecole de Rochefort],[pleiade],[symboliste],[symboliste],0
598,Alain_Robbe-Grillet,Nouveau Roman,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,8,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,7079,"[[Nouveau Roman], [Nouveau Roman], [symboliste...",[Nouveau Roman],[Nouveau Roman],[symboliste],[symboliste],0


In [89]:
AUT = auteurs[auteurs['del']==0]

In [91]:
AUT=AUT[['Mouvement','pred_tf_w','pred_tf_sum','pred_txt_w','pred_txt_s']]

In [126]:
AUT['test']=AUT['pred_tf_w'].apply(len)

<ipython-input-126-72f5baad327e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AUT['test']=AUT['pred_tf_w'].apply(len)


In [102]:
AUT['Mouvement'][0]  == AUT['pred_tf_w'][0][0]

True

In [142]:
AUT.index = [k for k in range(len(AUT))]

In [129]:
accuracy_tf_w = np.array([ int(AUT['Mouvement'][k] in AUT['pred_tf_w'][k]) for k in range(559)])

In [130]:
accuracy_tf_w.sum()/559

0.38640429338103754

In [133]:
accuracy_tf_sum = np.array([ int(AUT['Mouvement'][k] in AUT['pred_tf_sum'][k]) for k in range(559)])

In [134]:
accuracy_tf_sum.sum()/559

0.4525939177101968

In [135]:
accuracy_txt_w = np.array([ int(AUT['Mouvement'][k] in AUT['pred_txt_w'][k]) for k in range(559)])

In [136]:
accuracy_txt_w.sum()/559

0.06976744186046512

In [140]:
accuracy_txt_s = np.array([ int(AUT['Mouvement'][k] in AUT['pred_txt_s'][k]) for k in range(559)])

In [141]:
accuracy_txt_s.sum()/559

0.10912343470483005

In [144]:
auteurs.to_csv('auteurs_pred_1.csv')

In [145]:
AUT.to_csv('auteurs_pred2.csv')